DESCARGAR BLENDER

In [ ]:
#Download Blender from Repository - Run Once unless version changed

!wget https://download.blender.org/release/Blender4.4/blender-4.4.0-linux-x64.tar.xz

In [ ]:
#Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Copy Blender to gdrive - Run Once unless version changed
!cp /content/blender-4.3.0-linux-x64.tar.xz /content/drive/MyDrive/Blender/blender-4.3.0-linux-x64.tar.xz

In [ ]:
#install Blender
!tar xf blender-4.4.0-linux-x64.tar.xz

In [ ]:
#Copy Blender back to Colab
!cp /content/drive/MyDrive/Blender/blender-4.3.0-linux-x64.tar.xz /content/blender-4.3.0-linux-x64.tar.xz

In [ ]:
# 1. Descargar el complemento PCB2Blender
!wget -O pcb2blender_importer.zip "https://github.com/30350n/pcb2blender/releases/download/v2.17.2-k9.0-b4.2lts/pcb2blender_importer_v2-17-2_b4-4.zip"
# 2. Descomprimir el archivo descargado
!unzip -o pcb2blender_importer.zip -d /tmp/pcb2blender_importer

# 3. Instalar la dependencia `error_helper` (corrigiendo la versión)
!./blender-4.4.0-linux-x64/4.4/python/bin/python3.11 -m pip install /tmp/pcb2blender_importer/wheels/error_helper-*.whl

# Instalar Skia en el entorno de Python de Blender
!./blender-4.4.0-linux-x64/4.4/python/bin/python3.11 -m pip install skia-python

# 4. Crear el directorio de addons en Blender
!mkdir -p ~/.config/blender/4.4/scripts/addons/

# 5. Copiar el complemento al directorio de Blender
!cp -r /tmp/pcb2blender_importer ~/.config/blender/4.4/scripts/addons/pcb2blender

# 6. Crear un script para habilitar el addon automáticamente
!echo "import bpy; bpy.ops.preferences.addon_enable(module='pcb2blender'); bpy.ops.wm.save_userpref()" > enable_addon.py

# 7. Instalar la librería Pillow (requerida por el complemento)
!./blender-4.4.0-linux-x64/4.4/python/bin/python3.11 -m pip install Pillow

# 8. Ejecutar Blender en segundo plano para activar el addon
!./blender-4.4.0-linux-x64/blender --background --python enable_addon.py

# 9. Eliminar el script auxiliar después de habilitar el addon
!rm enable_addon.py

In [ ]:
#patch init_py bl_info missing
init_file = "/root/.config/blender/4.4/scripts/addons/pcb2blender/__init__.py"

bl_info_block = '''bl_info = {
    "name": "PCB2Blender",
    "author": "30350n",
    "version": (2, 17, 2),
    "blender": (4, 4, 0),
    "location": "File > Import",
    "description": "Import PCB data into Blender",
    "category": "Import-Export",
}

'''

with open(init_file, 'r') as f:
    content = f.read()

if "bl_info" not in content:
    print("Agregando bl_info al inicio del archivo...")
    with open(init_file, 'w') as f:
        f.write(bl_info_block + content)
else:
    print("bl_info ya está presente. No se hacen cambios.")

In [ ]:
%%writefile list_addons.py
import addon_utils
for mod in addon_utils.modules():
    print(mod.__name__)

In [ ]:
!./blender-4.4.0-linux-x64/blender -b -P list_addons.py

CONFIGURAR BLENDER

In [ ]:
#crea ventana config
import ipywidgets as widgets
from IPython.display import display

# Configuración del archivo
blend_file = widgets.Text(value='/content/scene.blend', description='Blend File:')
output_path = widgets.Text(value='/content/drive/MyDrive/Blender/Test1/Renders', description='Output Path:')
output_format = widgets.Dropdown(options=['PNG', 'JPEG', 'TIFF', 'OPEN_EXR'], value='PNG', description='Format:')

# Frames
start_frame = widgets.IntText(value=1, description='Start Frame:')
end_frame = widgets.IntText(value=250, description='End Frame:')

# Configuración Cycles
samples = widgets.IntSlider(value=128, min=1, max=4096, step=1, description='Samples:')
preview_samples = widgets.IntSlider(value=64, min=1, max=1024, step=1, description='Preview Samples:')
max_bounces = widgets.IntSlider(value=12, min=0, max=24, step=1, description='Max Bounces:')
diffuse_bounces = widgets.IntSlider(value=4, min=0, max=24, step=1, description='Diffuse:')
glossy_bounces = widgets.IntSlider(value=4, min=0, max=24, step=1, description='Glossy:')
transp_bounces = widgets.IntSlider(value=8, min=0, max=24, step=1, description='Transparent:')
noise_threshold = widgets.FloatSlider(value=0.01, min=0.0, max=0.1, step=0.001, description='Noise Thresh:')
use_denoising = widgets.Checkbox(value=True, description='Denoising')
adaptive_sampling = widgets.Checkbox(value=True, description='Adaptive Sampling')
adaptive_threshold = widgets.FloatSlider(value=0.01, min=0.001, max=0.1, step=0.001, description='Adaptive Thresh:')
device = widgets.Dropdown(options=['OPTIX', 'CUDA', 'HIP', 'CPU'], value='OPTIX', description='Device:')

# Mostrar todos los widgets
display(blend_file, output_path, output_format, start_frame, end_frame,
        samples, preview_samples, max_bounces, diffuse_bounces, glossy_bounces,
        transp_bounces, noise_threshold, use_denoising, adaptive_sampling,
        adaptive_threshold, device)

In [ ]:

# Crear script con configuraciones
config_script = f"""
import bpy

scene = bpy.context.scene
cycles = scene.cycles
scene.render.image_settings.file_format = '{output_format.value}'

# Configuración Cycles
cycles.samples = {samples.value}
cycles.preview_samples = {preview_samples.value}
cycles.max_bounces = {max_bounces.value}
cycles.diffuse_bounces = {diffuse_bounces.value}
cycles.glossy_bounces = {glossy_bounces.value}
cycles.transparent_max_bounces = {transp_bounces.value}
cycles.noise_threshold = {noise_threshold.value}
cycles.use_denoising = {use_denoising.value}
cycles.use_adaptive_sampling = {adaptive_sampling.value}
cycles.adaptive_threshold = {adaptive_threshold.value}

# Configuración de render
scene.frame_start = {start_frame.value}
scene.frame_end = {end_frame.value}
"""

# Guardar script
with open("cycles_config.py", "w") as f:
    f.write(config_script)

print("✔ Script cycles_config.py generado.")

RENDER

In [ ]:

# Ejecutar Blender con el archivo de configuración
!./blender-4.4.0-linux-x64/blender -b {blend_file.value} -noaudio -E 'CYCLES' \
-P cycles_config.py \
-o '{output_path.value}' \
-s {start_frame.value} -e {end_frame.value} -a -F '{output_format.value}' \
-- --cycles-device {device.value}

In [ ]:
#set the paths to Blender Files
filename = '/content/drive/MyDrive/Blender/AMP_huesito.blend'

In [ ]:
#Render
!./blender-4.4.0-linux-x64/blender -b $filename -noaudio -E 'CYCLES' -o '/content/drive/MyDrive/Blender/Test1/Renders/testRender_'  -s 240 -e 250 -a -F 'PNG' -- --cycles-device OPTIX

In [ ]:

#Render custom or some else
!./blender-4.4.0-linux-x64/blender -P /content/drive/MyDrive/Blender/GPU.py -b "$filename" --gpu-backend opengl --gpu-compilation-subprocesses 2 -E 'CYCLES' -o '/content/drive/MyDrive/Blender/Test1/Renders/testRender' -s 150 -e 175 -a -F 'FFMPEG' --render-format 'MPEG4' --ffmpeg-format 'MPEG4' --ffmpeg-video-bitrate 8000 --ffmpeg-audio-codec 'NONE' -- --cycles-device OPTIX --cycles-print-stats -t $(nproc)

Make video

In [ ]:

#generar Video
import cv2
import os

# Configuración
folder_path = "/content/drive/MyDrive/Blender/Test2/"  # Reemplaza con la ruta a tus imágenes
output_file = "output_video.mp4"
frame_rate = 20  # Frames por segundo

# Tamaño del video (puedes detectarlo automáticamente con la primera imagen)
first_image_path = os.path.join(folder_path, "Renders0001.png")
first_frame = cv2.imread(first_image_path)

if first_frame is None:
    raise FileNotFoundError(f"No se pudo leer la imagen: {first_image_path}")

height, width, layers = first_frame.shape
video_size = (width, height)

# Inicializa el codificador de video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Para formato .mp4
video_writer = cv2.VideoWriter(output_file, fourcc, frame_rate, video_size)

# Lee y escribe cada imagen
for i in range(1, 251):  # De 0001 a 0250
    filename = f"Renders{i:04d}.png"
    image_path = os.path.join(folder_path, filename)
    frame = cv2.imread(image_path)

    if frame is None:
        print(f"Advertencia: No se pudo leer {image_path}, se omite.")
        continue

    video_writer.write(frame)

# Finaliza
video_writer.release()
print(f"Video guardado como '{output_file}'")